In [1]:
# Cryptocurrency Analysis Script v002

import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2015, 1, 1).timestamp())  # Jan 1, 2015
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/indicators/investor_capitalization',
    'https://api.glassnode.com/v1/metrics/supply/current',
    'https://api.glassnode.com/v1/metrics/indicators/liveliness',
    'https://api.glassnode.com/v1/metrics/indicators/realized_profit_lth_account_based',
    'https://api.glassnode.com/v1/metrics/indicators/realized_loss_lth_account_based',
    'https://api.glassnode.com/v1/metrics/supply/profit_relative'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

def calculate_supplyinprofit_percentile(df, column='profit_relative', perc_window=1440, norm_window=1440, normalization='z-score'):
    # Calculate percentile
    percentile = df[column].rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

def calculate_aviv_percentile(df, columns=['investor_capitalization', 'current', 'liveliness', 'price_usd_close'], perc_window=1440, norm_window=1440, normalization='z-score'):
    # Calculate AVIV
    aviv = df[columns[2]] * df[columns[1]] * df[columns[3]] / df[columns[0]]
    
    # Calculate percentile
    percentile = aviv.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

def calculate_lthrealized_plratio_logpercentile(df, columns=['realized_profit_lth_account_based', 'realized_loss_lth_account_based'], perc_window=1440, norm_window=1440, normalization='z-score'):
    # Calculate LTH realized P/L ratio
    lth_realized_pl = df[columns[0]] / df[columns[1]]
    
    # Apply log transformation
    log_lth_realized_pl = np.log(lth_realized_pl)
    
    # Calculate percentile
    percentile = log_lth_realized_pl.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

# Apply the functions to the merged dataframe with maxmin normalization
merged_df['supplyinprofit_percentile_maxmin'] = calculate_supplyinprofit_percentile(merged_df, normalization='maxmin')
merged_df['aviv_percentile_maxmin'] = calculate_aviv_percentile(merged_df, normalization='maxmin')
merged_df['lthrealized_plratio_logpercentile_maxmin'] = calculate_lthrealized_plratio_logpercentile(merged_df, normalization='maxmin')

In [2]:
merged_df.tail()

,price_usd_close,investor_capitalization,current,liveliness,realized_profit_lth_account_based,realized_loss_lth_account_based,profit_relative,supplyinprofit_percentile_maxmin,aviv_percentile_maxmin,lthrealized_plratio_logpercentile_maxmin
t,,,,,,,,,,
2024-09-20,63122.179999,5.609648e+11,1.247090e+12,0.609330,8.661174e+08,3.314733e+06,0.849782,0.328103,0.491845,0.385257
2024-09-21,63486.442220,5.612448e+11,1.254313e+12,0.609242,1.463015e+08,2.491532e+06,0.861928,0.380884,0.498712,0.208623
2024-09-22,63567.370107,5.613931e+11,1.256892e+12,0.609146,9.060098e+07,1.748635e+06,0.864112,0.389444,0.502146,0.189152
2024-09-23,63322.551391,5.621972e+11,1.251965e+12,0.609135,5.941965e+08,3.979330e+06,0.853964,0.340942,0.490129,0.346314
2024-09-24,64338.712500,5.625841e+11,1.270186e+12,0.609074,2.149525e+08,6.797343e+06,0.880322,0.459344,0.512446,0.116829
